In [ ]:
from neutromeratio import *
from openmmtools.constants import kB
from simtk import unit
import numpy as np
import pickle
import mdtraj as md
import torchani
import torch
from tqdm import tqdm

platform = 'cpu'
device = torch.device(platform)
model = torchani.models.ANI1ccx()
model = model.to(device)
torch.set_num_threads(2)

In [ ]:
exp_results = pickle.load(open('../data/exp_results.pickle', 'rb'))

# specify the system you want to simulate
name = 'molDWRow_298'
#name = 'molDWRow_37'
#name = 'molDWRow_45'
#name = 'molDWRow_160'
#name = 'molDWRow_590'


from_mol_tautomer_idx = 1
to_mol_tautomer_idx = 2

# specify a run number
nr_of_runs = 1
# number of time steps
nr_of_mc_trials = 50

t1_smiles = exp_results[name]['t1-smiles']
t2_smiles = exp_results[name]['t2-smiles']

# generate both rdkit mol
mols = { 't1' : generate_rdkit_mol(t1_smiles), 't2' : generate_rdkit_mol(t2_smiles) }
from_mol = mols[f"t{from_mol_tautomer_idx}"]
to_mol = mols[f"t{to_mol_tautomer_idx}"]
ani_input = from_mol_to_ani_input(from_mol)



In [ ]:
display_mol(mols['t1'])

In [ ]:
display_mol(mols['t2'])

In [ ]:
# extract hydrogen donor idx and hydrogen idx for m2
donor = get_donor_atom_idx(from_mol, to_mol)

# extract hydrogen acceptor idx for to_mol
###################################
###################################
acceptor = 5
###################################
###################################

species = model.species_to_tensor(ani_input['atom_list']).to(device).unsqueeze(0)

energy_function = ANI1cxx_force_and_energy(device = device,
                                          model = model,
                                          species = species,
                                          platform = platform)

hydrogen_mover = Instantenous_MC_Mover(donor_idx = donor['donor'], 
                                       hydrogen_idx = donor['hydrogen_idx'], 
                                       acceptor_idx = acceptor, 
                                       atom_list = ani_input['atom_list'], 
                                       energy_function = energy_function)

langevin = LangevinDynamics(atom_list = ani_input['atom_list'],
                            device = device,
                            model = model,
                            temperature = 300*unit.kelvin,
                            force = energy_function)


# initial conditions: coordinates from example were given in units of angstroms   
x0 = np.array(ani_input['coord_list']) * unit.angstrom
print(f"Hydrogen {hydrogen_mover.hydrogen_idx} is moved from atom-idx {hydrogen_mover.donor_idx} to atom-idx {hydrogen_mover.acceptor_idx}.")


for run in range(1, nr_of_runs+1):
    performe_md_mc_protocoll(name = name, 
                             run = run,
                             x0 = x0,
                             nr_of_mc_trials = nr_of_mc_trials, 
                             hydrogen_mover = hydrogen_mover,
                            langevin_dynamics = langevin,
                            from_mol_tautomer_idx = from_mol_tautomer_idx,
                            to_mol_tautomer_idx = to_mol_tautomer_idx)

In [ ]:
generate_nglview_object(
    top_file=f"../data/structures/{name}/{name}_tautomer_{from_mol_tautomer_idx}.pdb",
    traj_file=f"../data/md_mc_sampling/{name}/{name}_t{from_mol_tautomer_idx}_run{run}_anicxx.dcd"
)